<a href="https://colab.research.google.com/github/kat-le/cmpe255-pycaret/blob/main/time_series_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret kaggle
!kaggle -v

In [ ]:
import pycaret
pycaret.__version__

In [ ]:
from google.colab import files
up = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle

In [ ]:
!mkdir -p data
!kaggle datasets download -d agajorte/detroit-daily-temperatures-with-artificial-warming -p data --unzip
!ls -lah data

In [ ]:
import pandas as pd
import glob, os

csvs = sorted(glob.glob(os.path.join("data", "**", "*.csv"), recursive=True))
csvs

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import glob, os

# Find the CSV file in the new data directory
csv_files = glob.glob(os.path.join("data", "*.csv"))
if not csv_files:
    raise FileNotFoundError("No CSV file found in data directory.")
new_csv_path = csv_files[0]


df = pd.read_csv(new_csv_path)

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset=['date'], inplace=True)
df = df.set_index('date').sort_index()
df = df['temperature']

df = df.asfreq('D', fill_value=None).ffill()

# Filter data for the years 2015 to 2017
df = df.loc['2015':'2017']

display(df.head())

In [ ]:
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import pandas as pd

df.plot()
plt.title('Daily Temperature Time Series (Detroit)')
plt.xlabel('Year')
plt.ylabel('Temperature in Celsius')
plt.savefig('daily_temperature_timeseries.png') # Add this line to save the plot
plt.show()

In [ ]:
from pycaret.time_series import *

s = setup(df, fh = 5, session_id = 123)

In [ ]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()

In [ ]:
type(exp)

In [ ]:
exp.setup(df, fh = 5, session_id = 123)

In [ ]:
check_stats()

In [ ]:
best = compare_models()

In [ ]:
exp.compare_models()

In [30]:
plot_model(best, plot = 'forecast')
plt.savefig('best_model_forecast.png')

<Figure size 800x550 with 0 Axes>

In [ ]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 18})

In [ ]:
plot_model(best, plot = 'residuals')
plt.savefig('best_model_residuals.png')
plt.show()

In [31]:
# Create the ARIMA model
arima_model = create_model('arima')

plot_model(arima_model, plot = 'forecast')
plt.savefig('arima_model_forecast.png')
plt.show()

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2017-11-09,1.1455,1.0518,5.4744,6.4003,0.7996,0.4780,-2.6485
1,2017-11-14,1.0390,0.9565,4.9799,5.8365,0.3565,0.4815,-11.0574
2,2017-11-19,0.6682,0.6047,3.2022,3.6882,0.3136,0.2614,-1.8946
Mean,NaT,0.9509,0.8710,4.5522,5.3083,0.4899,0.4070,-5.2001
SD,NaT,0.2046,0.1923,0.9757,1.1685,0.2197,0.1030,4.1531


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

<Figure size 800x550 with 0 Axes>

In [32]:
plot_model(arima_model, plot = 'residuals')
plt.savefig('arima_model_residuals.png')
plt.show()

<Figure size 800x550 with 0 Axes>

In [33]:
plot_model(arima_model, plot = 'forecast', data_kwargs = {'fh' : 40})
plt.savefig('arima_model_long_forecast.png')
plt.show()

<Figure size 800x550 with 0 Axes>

In [ ]:
holdout_pred = predict_model(arima_model)

In [ ]:
holdout_pred.head()

In [ ]:
predict_model(arima_model, fh = 40)

In [ ]:
save_model(arima_model, 'forecast_arima')

In [ ]:
loaded_best_pipeline = load_model('forecast_arima')
loaded_best_pipeline

# Task
Demonstrate the ARIMA model prediction in Gradio.

## Install gradio

### Subtask:
Install the gradio library.


**Reasoning**:
The subtask is to install the gradio library. I will use pip to install the library.



In [ ]:
!pip install gradio

## Define prediction function

### Subtask:
Create a Python function that takes future horizon as input and returns the predicted temperatures using the loaded ARIMA model.


**Reasoning**:
Define a Python function that utilizes the loaded ARIMA model to generate future temperature predictions based on a given forecast horizon.



In [ ]:
def forecast_temperature(future_horizon):
  """
  Generates future temperature predictions using the loaded ARIMA model.

  Args:
    future_horizon: The number of steps to forecast into the future.

  Returns:
    A pandas DataFrame containing the forecasted temperature values.
  """
  predictions = predict_model(loaded_best_pipeline, fh=future_horizon)
  return predictions

## Create gradio interface

### Subtask:
Build a Gradio interface using the prediction function and specify the input and output components.


**Reasoning**:
Build a Gradio interface using the prediction function and specify the input and output components.



In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=forecast_temperature,
    inputs=gr.Number(label="Future Horizon (days)"),
    outputs=gr.DataFrame(label="Predicted Temperatures")
)

## Launch gradio app

### Subtask:
Launch the Gradio interface to interact with the model.


**Reasoning**:
Launch the Gradio interface to allow interaction with the prediction function.



In [ ]:
iface.launch()

## Summary:

### Data Analysis Key Findings

* The `gradio` library was already installed in the environment before the process began.
* A Python function, `forecast_temperature`, was created to utilize a pre-loaded ARIMA model from PyCaret for making future temperature predictions.
* A Gradio interface was successfully built using `gradio.Interface`, connecting the `forecast_temperature` function with a numeric input for the future horizon and a DataFrame output for the predictions.
* The Gradio interface was launched, making it accessible via a public URL.

### Insights or Next Steps

* The launched Gradio interface provides a user-friendly way to demonstrate the ARIMA model's prediction capabilities without requiring coding knowledge.
* The next steps could involve embedding this Gradio interface within a larger application or sharing the public URL with others to allow interaction with the model.
